In [ ]:
from stmpy import Driver, Machine
from threading import Thread
import paho.mqtt.client as mqtt
import json
from stmpy import Machine, Driver
import ipywidgets as widgets
from IPython.display import display
import random



class Teacher:
    def __init__(self):
        self.button_ask = widgets.Button(description="Ask")
        self.button_ask.on_click(self.on_button_ask)
        self.answered = ""
        self.message = ""
        # text field
        self.text = widgets.Text(value='', placeholder='', description='String:', disabled=False)
        display(self.text, self.button_ask)



# initial transition
t0 = {'source': 'initial',
      'target': 'idle'}

# transitions
t1 = {'trigger':'ask', 
      'source':'idle', 
      'target':'asking'}
t2 = {'trigger':'t', 
      'source':'asking', 
      'target':'idle'}
t3 = {'trigger':'t', 
      'source':'received', 
      'target':'idle'}
t4 = {'trigger':'message', 
      'source':'asking', 
      'target':'received'}
t5 = {'trigger':'reset', 
      'source':'asking', 
      'target':'idle'}

 
# the states:
idle = {'name': 'idle',
       'entry': 'on_idle'}

asking = {'name': 'asking',
        'entry': 'on_asking; start_timer("t", 20000)'}

received = {'name': 'received',
        'entry': 'on_received; stop_timer("t")'}
